In [ ]:
import pandas as pd
import numpy as np
from keras.models import Sequential,Model
from keras.layers import Dense, Dropout, Activation, Flatten, LSTM, TimeDistributed, RepeatVector,Embedding,concatenate,Input
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import joblib
import multiprocessing
from keras import backend as K
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from data_tool import label_encode,label_decode,eachAccu
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

seed =999
fea_dim = '2368' # 512/2048/2368/832
fea_type = 'BOW'
X = joblib.load('./fea_sel/{}/{}_train-{}.pkl'.format(fea_type,fea_type.lower(),fea_dim))
hashtag = [] 
label = joblib.load('./fea_sel/label/label8.pkl')
label_encoder = LabelEncoder()
label_encoder.fit(label)
y = label_encode(label_encoder,label)
NUM_CLASSES = 8
NUM_LSTM_UNITS = 128 #output dimension of lstm 
max_features = 2368
x_train,x_valid,y_train,y_valid = train_test_split(X,y, random_state =seed,test_size = 0.2 )
BZ= [512]

In [ ]:
def lstm(x_train,y_train,x_valid,y_valid,max_features,bz):
    # K.tensorflow_backend._get_available_gpus()
    # cfg = tf.ConfigProto()
    # cfg.gpu_options.allow_growth = True
    # with tf.device('/gpu:1'):
    #     with tf.Session(config = cfg) as sess:       
    model = Sequential()
    # model.add(Input((X.shape[1],),sparse = True))
    model.add(Embedding(max_features, 128))
    model.add(LSTM(128, input_shape = (X.shape[1],),return_sequences=True)) #dropout=0.2, recurrent_dropout=0.2,
    model.add(Dropout(0.4))
    model.add(LSTM(16, activation='tanh'))
    model.add(Dropout(0.4))
    model.add(Dense(8, activation='softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    model.summary()
    return model

In [ ]:
for bz in BZ:
    # pool = multiprocessing.Pool(processes=16)
    epo=10
    model = lstm(x_train,y_train,x_valid,y_valid,max_features,bz)
    print('Train...')
    model.fit(x_train, y_train,
              batch_size=bz,
              epochs=epo,
              validation_data=(x_valid, y_valid))
    score, acc = model.evaluate(x_valid, y_valid,batch_size=bz)   
    # sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
    # sess.run()
    #=================== TEST DATA===============================
    '''
    pred = model.predict(test_data)
    pred = label_encode(label_encoder,pred)
    gnd = label_encode(label_encoder,y_valid)
    
    f1_list = eachAccu(gnd,pred)
    f1_overall = f1_score(pred,gnd,average='micro')
    print('\nf1_overall:{:.4}'.format(f1_overall))
    cm = confusion_matrix(gnd, pred)
    plot_confusion_matrix(cm , class_names=['sadness' ,'disgust' ,'anticipation', 'joy' ,'trust' ,'anger', 'fear','surprise'])
    test_data = joblib.load('./fea_sel/BOW/bow_test-2368.pkl')
    '''